In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/units_2020_dummies.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461348 entries, 0 to 461347
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   transaction_id        461348 non-null  object 
 1   procedure_id          461348 non-null  int64  
 2   trans_group_id        461348 non-null  int64  
 3   trans_group_en        461348 non-null  object 
 4   procedure_name_en     461348 non-null  object 
 5   instance_date         461348 non-null  object 
 6   property_type_id      461348 non-null  int64  
 7   property_type_en      461348 non-null  object 
 8   property_sub_type_id  461348 non-null  float64
 9   property_sub_type_en  461348 non-null  object 
 10  property_usage_en     461348 non-null  object 
 11  reg_type_id           461348 non-null  int64  
 12  reg_type_en           461348 non-null  object 
 13  area_id               461348 non-null  int64  
 14  area_name_en          461348 non-null  object 
 15  

# unnecessary

In [ ]:
df['instance_year'] = pd.to_numeric(df['instance_year'], errors='coerce')
df = df.dropna(subset=['instance_year'])
df['instance_year'] = df['instance_year'].astype(int)


In [ ]:
start_year = df['instance_year'].min()
end_year = df['instance_year'].max()

print(f"Start year: {start_year}")
print(f"End year: {end_year}")


Start year: 2020.0
End year: 2025.0


In [ ]:
bins = list(range(start_year, end_year + 5, 5))
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]


In [ ]:
df['Year_Range'] = pd.cut(df['instance_year'], bins=bins, labels=labels, right=False)


In [ ]:
dummy_df = pd.get_dummies(df['Year_Range'], prefix='YearBin').astype(int)




In [ ]:
df = pd.concat([df, dummy_df], axis=1)


In [ ]:
df.head()


,transaction_id,procedure_id,trans_group_id,trans_group_en,procedure_name_en,instance_date,property_type_id,property_type_en,property_sub_type_id,property_sub_type_en,...,meter_rent_price,no_of_parties_role_1,no_of_parties_role_2,no_of_parties_role_3,instance_year,instance_month,instance_day,sector_name,Year_Range,YearBin_2020-2024
0,1-11-2023-32261,11,1,Sales,Sell,2023-10-09,2,Building,NaN,NaN,...,NaN,1.0,2.0,0.0,2023,10.0,9.0,West,2020-2024,1
1,1-11-2023-33947,11,1,Sales,Sell,2023-10-23,2,Building,NaN,NaN,...,NaN,1.0,2.0,0.0,2023,10.0,23.0,West,2020-2024,1
2,1-11-2021-10599,11,1,Sales,Sell,2021-06-23,4,Villa,NaN,NaN,...,NaN,3.0,1.0,0.0,2021,6.0,23.0,West,2020-2024,1
3,1-11-2024-26324,11,1,Sales,Sell,2024-07-17,1,Land,NaN,NaN,...,NaN,1.0,1.0,0.0,2024,7.0,17.0,West,2020-2024,1
4,1-11-2022-19764,11,1,Sales,Sell,2022-08-23,4,Villa,NaN,NaN,...,NaN,25.0,1.0,0.0,2022,8.0,23.0,North_west,2020-2024,1


In [ ]:
# prompt: write a code to get the unique values in particular column 'instance_date'

unique_instance_dates = df['instance_year'].unique()
unique_instance_dates


array([2023, 2021, 2024, 2022, 2020, 2025])

# dummies

## property_usage_en

In [ ]:
value_counts = df['property_usage_en'].value_counts()
percentage = df['property_usage_en'].value_counts(normalize=True) * 100

In [ ]:
property_usage_summary = pd.DataFrame({
    'Category': value_counts.index,
    'Count': value_counts.values,
    'Percentage': percentage.values.round(2)
})

# Step 5: Display the summary
property_usage_summary

,Category,Count,Percentage
0,Residential,428203,92.82
1,Commercial,20672,4.48
2,Hospitality,12405,2.69
3,Industrial,51,0.01
4,Other,11,0.00
5,Storage,6,0.00


In [ ]:
df['property_usage_en'] = df['property_usage_en'].astype(str).str.strip().str.lower()

In [ ]:
df['usage_residential'] = df['property_usage_en'].apply(lambda x: 1 if x == 'residential' else 0)
df['usage_hospitality'] = df['property_usage_en'].apply(lambda x: 1 if x == 'hospitality' else 0)



##reg_type_en dummies

In [ ]:
# prompt: f['reg_type_en'] = df['reg_type_en'].astype(str).str.strip().str.lower()
# df['Off_Plan_Reg'] = df['reg_type_en'].apply(lambda x: 1 if 'off-plan properties' in x else 0) this I tried again its coming 0 only

# Key change: Handle potential NaN values and case-insensitive comparison
df['reg_type_en'] = df['reg_type_en'].astype(str).str.strip().str.lower()
df['Off_Plan_Reg'] = df['reg_type_en'].apply(lambda x: 1 if 'off-plan properties' in x else 0)

# Display relevant columns for verification
df[['reg_type_en','Off_Plan_Reg']]



,reg_type_en,Off_Plan_Reg
0,existing properties,0
1,off-plan properties,1
2,off-plan properties,1
3,off-plan properties,1
4,existing properties,0
...,...,...
461343,existing properties,0
461344,existing properties,0
461345,existing properties,0
461346,off-plan properties,1


In [ ]:
df['reg_type_en'].unique()

array(['existing properties', 'off-plan properties'], dtype=object)

In [ ]:
df[['reg_type_en','Off_Plan_Reg']].sample(5)

,reg_type_en,Off_Plan_Reg
37003,existing properties,0
276575,existing properties,0
176790,off-plan properties,1
138755,off-plan properties,1
208256,existing properties,0


##trans_group_en dummies

In [ ]:
df['trans_group_en'] = df['trans_group_en'].astype(str).str.strip().str.lower()

In [ ]:
df['trans_group_gifts'] = df['trans_group_en'].apply(lambda x: 1 if x == 'gifts' else 0)
df['trans_group_sales'] = df['trans_group_en'].apply(lambda x: 1 if x == 'sales' else 0)


In [ ]:
df[['trans_group_en','trans_group_gifts','trans_group_sales']].sample(10)

,trans_group_en,trans_group_gifts,trans_group_sales
168110,sales,0,1
313324,sales,0,1
432877,sales,0,1
409541,sales,0,1
299768,sales,0,1
363436,sales,0,1
309065,sales,0,1
417641,mortgages,0,0
124551,sales,0,1
205370,sales,0,1


##property_type_en dummies

In [ ]:
df['property_type_en'] = df['property_type_en'].astype(str).str.strip().str.lower()

# Create the new columns based on property type
df['Property_Building'] = df['property_type_en'].apply(lambda x: 1 if x == 'building' else 0)
df['Property_Unit'] = df['property_type_en'].apply(lambda x: 1 if x == 'unit' else 0)
df['Property_Villa'] = df['property_type_en'].apply(lambda x: 1 if x == 'villa' else 0)



In [ ]:
# Display a sample to verify the changes
print(df[['property_type_en', 'Property_Building', 'Property_Unit', 'Property_Villa']].sample(20))

       property_type_en  Property_Building  Property_Unit  Property_Villa
440356             unit                  0              1               0
30288              unit                  0              1               0
193546             unit                  0              1               0
421934             unit                  0              1               0
391387             unit                  0              1               0
215101             unit                  0              1               0
85196              unit                  0              1               0
72424              unit                  0              1               0
419509             unit                  0              1               0
219404             unit                  0              1               0
424274             unit                  0              1               0
345913             unit                  0              1               0
129699             unit               

##nearest_landmark_en dummies

In [ ]:
# prompt: for every nearest_landmark_en it is coming 0 see the above code and give me the solution

# Check unique values in 'nearest_landmark_en' to identify potential issues
print(df['nearest_landmark_en'].unique())

# Improved handling of NaN values and string comparisons
df['nearest_landmark_en'] = df['nearest_landmark_en'].astype(str).str.strip().str.lower()
df['Fam_landmarks'] = df['nearest_landmark_en'].apply(lambda x: 1 if x in ['jabel ali','burj khalifa', 'global village', 'downtown dubai', 'dubai international airport', 'burj al arab'] else 0)

# Display a sample to verify the changes
print(df[['nearest_landmark_en','Fam_landmarks']].sample(50))


['Burj Khalifa' 'Dubai International Airport'
 'Sports City Swimming Academy' 'Burj Al Arab' nan 'Downtown Dubai'
 'Motor City' 'Expo 2020 Site' 'Dubai Parks and Resorts' 'Global Village'
 'IMG World Adventures' 'Dubai Cycling Course'
 'Al Makhtoum International Airport']
                 nearest_landmark_en  Fam_landmarks
437797                downtown dubai              1
418562                    motor city              0
349998                           nan              0
455917  sports city swimming academy              0
283199                downtown dubai              1
342237                expo 2020 site              0
126210          img world adventures              0
13593                 downtown dubai              1
415164                           nan              0
108422  sports city swimming academy              0
250917  sports city swimming academy              0
235022                    motor city              0
259210                  burj khalifa              1

##nearest_mall_en dummies

In [ ]:
df['nearest_mall_en'] = df['nearest_mall_en'].astype(str).str.strip().str.lower()


In [ ]:
df['close_DubaiMall'] = df['nearest_mall_en'].apply(lambda x: 1 if x == 'dubai mall' else 0)
df['Close_Marina_EmiratesMall'] = df['nearest_mall_en'].apply(lambda x: 1 if x in ['mall of the emirates','marina mall'] else 0)
df['Close_MIRDIF_IBNMall'] = df['nearest_mall_en'].apply(lambda x: 1 if x in ['city centre mirdif','ibn-e-battuta mall'] else 0)

In [ ]:
df[['nearest_mall_en','close_DubaiMall','Close_Marina_EmiratesMall','Close_MIRDIF_IBNMall']].sample(10)

,nearest_mall_en,close_DubaiMall,Close_Marina_EmiratesMall,Close_MIRDIF_IBNMall
413250,ibn-e-battuta mall,0,0,1
47731,nan,0,0,0
142797,marina mall,0,1,0
232526,dubai mall,1,0,0
459500,nan,0,0,0
363272,mall of the emirates,0,1,0
222345,nan,0,0,0
315099,nan,0,0,0
340412,dubai mall,1,0,0
378406,nan,0,0,0


##rooms_en dumnmies

In [ ]:
df['rooms_en'] = df['rooms_en'].astype(str).str.strip().str.lower()

In [ ]:
df['Rooms_3_4'] = df['rooms_en'].isin(['3 b/r', '4 b/r']).astype(int)
df['Rooms_moreThan_4'] = df['rooms_en'].isin(['5 b/r', '6 b/r', 'penthouse']).astype(int)
df['Rooms_Studio_Single'] = df['rooms_en'].isin(['single room', 'studio']).astype(int)
df['Rooms_Commercial'] = df['rooms_en'].isin(['office', 'shop', 'store', 'gym']).astype(int)

In [ ]:
df[['rooms_en','Rooms_3_4', 'Rooms_moreThan_4', 'Rooms_Studio_Single', 'Rooms_Commercial']].sample(10)

,rooms_en,Rooms_3_4,Rooms_moreThan_4,Rooms_Studio_Single,Rooms_Commercial
342231,2 b/r,0,0,0,0
129553,studio,0,0,1,0
317015,1 b/r,0,0,0,0
217882,2 b/r,0,0,0,0
392330,1 b/r,0,0,0,0
133463,2 b/r,0,0,0,0
226543,2 b/r,0,0,0,0
252228,1 b/r,0,0,0,0
171609,studio,0,0,1,0
321385,1 b/r,0,0,0,0


# saving as csv

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461348 entries, 0 to 461347
Data columns (total 56 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   transaction_id             461348 non-null  object 
 1   procedure_id               461348 non-null  int64  
 2   trans_group_id             461348 non-null  int64  
 3   trans_group_en             461348 non-null  object 
 4   procedure_name_en          461348 non-null  object 
 5   instance_date              461348 non-null  object 
 6   property_type_id           461348 non-null  int64  
 7   property_type_en           461348 non-null  object 
 8   property_sub_type_id       461348 non-null  float64
 9   property_sub_type_en       461348 non-null  object 
 10  property_usage_en          461348 non-null  object 
 11  reg_type_id                461348 non-null  int64  
 12  reg_type_en                461348 non-null  object 
 13  area_id                    46

In [ ]:
df.to_csv('/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/units_2020_dummies.csv',index=False)

In [ ]:
dir(df)

['Close_MIRDIF_IBNMall',
 'Close_Marina_EmiratesMall',
 'Fam_landmarks',
 'Off_Plan_Reg',
 'Property_Building',
 'Property_Unit',
 'Property_Villa',
 'Rooms_3_4',
 'Rooms_Commercial',
 'Rooms_Studio_Single',
 'Rooms_moreThan_4',
 'T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__arrow_c_stream__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__dataframe_consortium_standard__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__

In [ ]:
df.shape[1]

52